## Movielens - 100K Dataset

MovieLens 100K dataset has been a standard dataset used for benchmarking recommender systems for more than 20 years now and hence this provides a good point to start our learning journey for recommender systems. For non commercial personalised recommendations for movies you can check out the website: https://movielens.org/

The data was collected through the MovieLens web site (movielens.umn.edu) during the seven-month period from September 19th, 1997 through April 22nd, 1998. This data has been cleaned up - users who had less than 20 ratings or did not have complete demographic information were removed from this data set. 

## Data Description


**Ratings**    -- The full u data set, 100000 ratings by 943 users on 1682 items.
              Each user has rated at least 20 movies.  Users and items are
              numbered consecutively from 1. This is a comma separated list of 
	         user id | item id | rating | timestamp. 
              The time stamps are unix seconds since 1/1/1970 UTC   


**Movie Information**   -- Information about the items (movies); this is a comma separated
              list of
              movie id | movie title | release date | unknown | Action | Adventure | Animation |
              Children's | Comedy | Crime | Documentary | Drama | Fantasy |
              Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
              Thriller | War | Western |
              The last 19 fields are the genres, a 1 indicates the movie
              is of that genre, a 0 indicates it is not; movies can be in
              several genres at once.


**User Demographics**    -- Demographic information about the users; this is a comma
              separated list of
              user id | age | gender | occupation | zip code

## Table of Content

[1. Reading Dataset](#Reading-Dataset)

[2. Basic Exploration](#Basic-Exploration)

[3. Merging Movie information to ratings dataframe](#merge)

[4. Non Personalised Recommender Systems using average ratings](#average)

[5. Non Personalised Recommender Systems using number of ratings or rating count](#ratingcount)

[6. Non Personalised Recommender Systems using count of ratings 4 and above](#4ratings)

[7. Weak Personalisation using Gender Information](#weakratings)

[8. What's Next](#whatsnext)





## 1. Reading Dataset <a class="anchor" id="Reading-Dataset"></a>


In [1]:
import pandas as pd
import numpy as np

C:\Anaconda\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [26]:
#Reading users file:
users = pd.read_csv('user_demographics.csv')

#Reading ratings file:
ratings= pd.read_csv('ratings.csv')

#Reading items file:
movie_info = pd.read_csv('movie_info.csv')

## 2. Basic Exploration <a class="anchor" id="Basic-Exploration"></a>

Let us look at each table to understand what we are dealing with here

### Exploring user data

In [27]:
# shape of the users data
print(users.shape)
# view the users data
users.head()

(943, 5)


,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [28]:
pd.isnull(users).sum()

user_id       0
age           0
sex           0
occupation    0
zip_code      0
dtype: int64

So, we have 943 users in the dataset and each user has 5 features, i.e. user_ID, age, sex, occupation and zip_code. We have no missing values in the user data. Now let’s look at the ratings file.

### Exploring ratings data

In [29]:
# shape of the data
print(ratings.shape)
# view the ratings data
ratings.head()

(100000, 4)


,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [30]:
ratings[(ratings['user_id'] == 1)&(ratings['movie_id'] == 100)]

,user_id,movie_id,rating,unix_timestamp
17672,1,100,5,878543541


In [31]:
pd.isnull(ratings).sum()

user_id           0
movie_id          0
rating            0
unix_timestamp    0
dtype: int64

We have 100k ratings for different user and movie combinations. Again there are no missing values here. Now lets examine the items file.

### Exploring Movie Information data

In [32]:
# shape of the data
print(movie_info.shape)
# view the items file
movie_info.head()

(4803, 6)


,id,title,keywords,cast,genres,director
0,19995,Avatar,culture clash future space war space colony so...,Sam Worthington Zoe Saldana Sigourney Weaver S...,Action Adventure Fantasy Science Fiction,James Cameron
1,285,Pirates of the Caribbean: At World's End,ocean drug abuse exotic island east india trad...,Johnny Depp Orlando Bloom Keira Knightley Stel...,Adventure Fantasy Action,Gore Verbinski
2,206647,Spectre,spy based on novel secret agent sequel mi6,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Action Adventure Crime,Sam Mendes
3,49026,The Dark Knight Rises,dc comics crime fighter terrorist secret ident...,Christian Bale Michael Caine Gary Oldman Anne ...,Action Crime Drama Thriller,Christopher Nolan
4,49529,John Carter,based on novel mars medallion space travel pri...,Taylor Kitsch Lynn Collins Samantha Morton Wil...,Action Adventure Science Fiction,Andrew Stanton


In [33]:
# Check missing values in movie information
pd.isnull(movie_info).sum()

id            0
title         0
keywords    412
cast         43
genres       28
director     30
dtype: int64

This dataset contains attributes of 1682 movies. There are 24 columns out of which last 19 columns specify the genre of a particular movie. These are binary columns, i.e., a value of 1 denotes that the movie belongs to that genre, and 0 otherwise.

We have release date missing for only 1 movie in the dataset and rest of the variables do not have any missing value

## 3.  Merging Movie information to ratings dataframe <a class="anchor" id="merge"></a>

The movie names are contained in a separate file. Let's merge that data with ratings and store it in ratings dataframe. The idea is to bring movie title information in ratings dataframe as it would be useful later on

In [34]:
ratings = ratings.merge(movie_info[['id','title']], how='left', left_on = 'movie_id', right_on = 'id')

In [35]:
ratings.head()

,user_id,movie_id,rating,unix_timestamp,id,title
0,196,242,3,881250949,242.0,The Godfather: Part III
1,186,302,3,891717742,302.0,Swimming Pool
2,22,377,1,878887116,377.0,A Nightmare on Elm Street
3,244,51,2,880606923,NaN,NaN
4,166,346,1,886397596,346.0,Seven Samurai


Lets also combine movie id and movie title separated by ': ' and store it in a new column named movie

In [36]:
ratings['movie'] = ratings['movie_id'].map(str) + str(': ') + ratings['title'].map(str)

In [37]:
ratings.head()

,user_id,movie_id,rating,unix_timestamp,id,title,movie
0,196,242,3,881250949,242.0,The Godfather: Part III,242: The Godfather: Part III
1,186,302,3,891717742,302.0,Swimming Pool,302: Swimming Pool
2,22,377,1,878887116,377.0,A Nightmare on Elm Street,377: A Nightmare on Elm Street
3,244,51,2,880606923,NaN,NaN,51: nan
4,166,346,1,886397596,346.0,Seven Samurai,346: Seven Samurai


In [38]:
ratings.columns

Index(['user_id', 'movie_id', 'rating', 'unix_timestamp', 'id', 'title',
       'movie'],
      dtype='object')

Keeping the columns movie, user_id and rating in the ratings dataframe and drop all others

In [40]:
ratings = ratings.drop(['movie_id', 'title', 'unix_timestamp'], axis = 1)

In [41]:
ratings = ratings[['user_id','movie','rating']]

For using non personalised recommender systems we are only interested in popular movies so we keep movies with atleast 100 ratings in the dataframe and drop the rest

In [44]:
movie_counts = ratings['movie'].value_counts()
ratings = ratings[(ratings['movie'].isin(movie_counts[movie_counts >= 100].index))]

Next, we create a user item matrix using Pandas Pivot Function such that users are in the index and each movie is represented by a separate column**
- Merge user data with ratings data
- Create user movie matrix using user ids as rows and movies as columns & name it 'user_movie_matrix'

User|Star Wars|Fargo|Contact
-|-|-|-
User 1|2|3|5
User 2|4|NA|NA
User 3|5|4|5
User 4|3|NA|2

In [45]:
ratings.head()

,user_id,movie,rating
0,196,242: The Godfather: Part III,3
1,186,302: Swimming Pool,3
4,166,346: Seven Samurai,1
5,298,474: nan,4
6,115,265: nan,2


In [46]:
n_users = ratings.user_id.unique().shape[0]
n_items = ratings.movie.unique().shape[0]

In [47]:
n_users, n_items

(943, 338)

In [48]:
user_movie_matrix = ratings.pivot(index = 'user_id', columns = 'movie', values = 'rating')

In [49]:
user_movie_matrix

movie,"100: Lock, Stock and Two Smoking Barrels",1012: nan,1016: nan,1028: nan,1047: nan,109: nan,111: Scarface,116: Match Point,117: The Untouchables,118: Charlie and the Chocolate Factory,...,928: Gremlins 2: The New Batch,92: nan,93: nan,94: nan,95: Armageddon,96: Beverly Hills Cop II,97: nan,98: Gladiator,99: nan,9: nan
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,5.0,5.0,3.0,3.0,3.0,...,NaN,3.0,5.0,2.0,4.0,5.0,3.0,4.0,3.0,5.0
2,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,4.0,NaN,NaN,3.0,3.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
940,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,5.0,NaN,4.0,NaN,3.0
941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
ratings = ratings.merge(users[['user_id','sex']], how = 'left', on = 'user_id')
ratings = ratings[['user_id','sex','movie','rating']]

In [53]:
ratings.head()

,user_id,sex,movie,rating
0,196,M,242: The Godfather: Part III,3
1,186,F,302: Swimming Pool,3
2,166,M,346: Seven Samurai,1
3,298,M,474: nan,4
4,115,M,265: nan,2


## 4. Non Personalised Recommender Systems using average ratings <a class="anchor" id="average"></a>

Here we calculate the mean rating for each movie, order with the highest rating listed first, and find the top five movies

In [22]:
user_movie_matrix.mean(axis=0).sort_values(ascending=False).head(5)

movie
408: Close Shave, A (1995)              4.491071
318: Schindler's List (1993)            4.466443
169: Wrong Trousers, The (1993)         4.466102
483: Casablanca (1942)                  4.456790
64: Shawshank Redemption, The (1994)    4.445230
dtype: float64

Interestingly average rating placed close shave which is a short animated movie at the top altough it is not very popular

## 5. Non Personalised Recommender Systems using number of ratings or rating count <a class="anchor" id="ratingcount"></a>

Here we count the number of ratings for each movie, order with the most number of ratings first, and find the top five.

In [23]:
user_movie_matrix.count(axis=0).sort_values(ascending=False).head(5)

movie
50: Star Wars (1977)              583
258: Contact (1997)               509
100: Fargo (1996)                 508
181: Return of the Jedi (1983)    507
294: Liar Liar (1997)             485
dtype: int64

We see here that average rating listed very different movies from when we tried to use rating count instead. Here we have more common movies as expected such as Star Wars, Fargo, Return of the Jedi etc.

## 6. Non Personalised Recommender Systems using count of ratings 4 and above <a class="anchor" id="4ratings"></a>

Here we calculate the percentage of ratings for each movie that are 4 or higher and order with the highest percentage first. Notice that the three different measures of "best" reflect different priorities and give different results; this should help you see why you need to be thoughtful about what metrics you use.

In [24]:
user_movie_matrix.apply(pd.value_counts)

movie,100: Fargo (1996),1012: Private Parts (1997),1016: Con Air (1997),1028: Grumpier Old Men (1995),1047: Multiplicity (1996),109: Mystery Science Theater 3000: The Movie (1996),"111: Truth About Cats & Dogs, The (1996)",116: Cold Comfort Farm (1995),"117: Rock, The (1996)",118: Twister (1996),...,"928: Craft, The (1996)",92: True Romance (1993),93: Welcome to the Dollhouse (1995),94: Home Alone (1990),95: Aladdin (1992),96: Terminator 2: Judgment Day (1991),97: Dances with Wolves (1990),"98: Silence of the Lambs, The (1991)",99: Snow White and the Seven Dwarfs (1937),9: Dead Man Walking (1995)
1.0,14,8,8,10,14,8,14,5,9,20,...,9,6,6,7,2,6,6,6,1,11
2.0,18,7,11,40,33,16,18,9,37,59,...,16,4,9,35,15,20,26,10,12,18
3.0,70,22,50,47,54,39,92,25,92,87,...,45,30,20,46,58,43,57,30,59,59
4.0,179,50,46,36,27,46,118,50,163,92,...,22,48,52,37,91,123,93,163,64,114
5.0,227,13,22,15,6,21,30,36,77,35,...,12,16,25,12,53,103,74,181,36,97


In [25]:
df_four = user_movie_matrix.apply(lambda x: x[x>=4]).count(axis=0) / user_movie_matrix.apply(lambda x: x).count(axis=0)
df_four.sort_values(ascending = False).head(5)

movie
479: Vertigo (1958)                     0.905028
64: Shawshank Redemption, The (1994)    0.901060
408: Close Shave, A (1995)              0.892857
169: Wrong Trousers, The (1993)         0.889831
318: Schindler's List (1993)            0.889262
dtype: float64

- The value 0.90 in this context most likely represents the proportion of users who rated a particular movie 4 or higher (on a rating scale where 4 is likely considered a "good" rating).

## 7. Weak Personalisation using Gender Information <a class="anchor" id="weakratings"></a>

Till now we have seen absolute non personalised recommendations, the first step towards personalisation can be taken here by using the gender of the user.
We first recompute the mean rating for each movie separately for males and for females and then calculate the overall mean rating (across all ratings) for males and females. 
Then we find out the 5 movies that have the greatest differences (one where men are most above women, and one where women are most above men) along with the differences in average.
The steps to use here are:
- Add the gender information from user dataframe
- Calculate Difference in average of men and women

In [61]:
user_movie_matrix['sex'] = list(users['sex'])

In [62]:
# Exclude the 'sex' column from the DataFrame before calculating the mean
df_m = user_movie_matrix[user_movie_matrix['sex'] == 'M'].drop(columns=['sex'])
df_m_mean = df_m.mean(axis=0).sort_values(ascending=False)

df_f = user_movie_matrix[user_movie_matrix['sex'] == 'F'].drop(columns=['sex'])
df_f_mean = df_f.mean(axis=0).sort_values(ascending=False)

# Calculate the difference between female and male users' average ratings
dif_g = df_f_mean - df_m_mean
dif_g.sort_values(ascending=False)


movie
476: nan                                     0.748951
485: nan                                     0.635686
29: nan                                      0.635452
38: Eternal Sunshine of the Spotless Mind    0.625616
451: Leaving Las Vegas                       0.612793
                                               ...   
199: Star Trek: First Contact               -0.621978
554: nan                                    -0.664553
156: nan                                    -0.693785
92: nan                                     -0.727273
177: The Fisher King                        -0.762913
Length: 338, dtype: float64

- 0.625616: Female users, on average, rated this movie 0.625616 higher than male users did.

Here we have done simple comparison for average ratings for male and female users for each movie and calculated the difference. This is personalisation at some level as we are taking the demographic of the user into consideration.

You could also compute the % of ratings 4+ separately for males and females and again find out the movies with the largest difference in both directions positive and negative

In [59]:
df_m_four = df_m.apply(lambda x: x[x>=4]).count(axis=0).sort_values(ascending=False) / df_m.apply(lambda x: x).count(axis=0)
df_f_four = df_f.apply(lambda x: x[x>=4]).count(axis=0).sort_values(ascending=False) / df_f.apply(lambda x: x).count(axis=0)

In [60]:
dif_g = df_f_four - df_m_four
dif_g.sort_values(ascending=False)

movie
476: nan                                     0.344353
38: Eternal Sunshine of the Spotless Mind    0.306934
225: nan                                     0.298450
485: nan                                     0.279403
402: nan                                     0.274242
                                               ...   
523: nan                                    -0.297573
331: Jurassic Park III                      -0.331281
156: nan                                    -0.355367
92: nan                                     -0.357955
205: Hotel Rwanda                           -0.358178
Length: 338, dtype: float64

Clearly some new movies have popped up here. We saw how easy it was to create non personalised recommender systems for movies and we did not need to remove old movies as when it comes to movies users are not that worried about how old is a particular movie rather they are interested in actors, genre etc.

### 8. What's Next <a class="anchor" id="whatsnext"></a>
- Try using other demographics such as occupation or age to see what kind of recommendations does it produce
- Also try both 4+ ratings technique and average ratings
- Next we will learn about personalisation and see how useful can that be for recommending movies